In [1]:
import pandas as pd
import numpy as np
from datetime import date
import glob
import os
import ipysheet

# NMD File

In [2]:
def extract_string_for_tag(string, tag):
    i = string.find('<' + tag)
    f = string.find('</' + tag + '>') 
    if f<0:
        f = string.find('/>')
        end_tag_len = 2
    else:
        end_tag_len = 3 + len(tag)
    return string[i:f], f+end_tag_len

def convert_value(value):
    value = value.strip()
    if len(value) == 0:
        return value
    elif value[0] in ['"', "'"] and value[-1] in ['"', "'"] and value[0] == value[-1]:
        value = value[1:-1]
    for convert_func in [int, float]:
        try:
            value = convert_func(value)
        except:
            pass
        else:
            return value

    return value

def parse_parametets(parameter_list):
    result = {}
    key = 'content'
    for parameter in parameter_list:
        if '=' in parameter:
            key = parameter.split('=')[0]
            value = "=".join(parameter.split('=')[1:])
            result[key] = convert_value(value)
        elif key not in result:
            result[key] = parameter
        else:
            result[key] += " " + parameter
    return result

def parse_tag(string, tag):
    open_i = string[1:].find('<')
    close_i = string[1:].find('>')
    close_direct = string[1:].find('/>')
    result = {}
    if open_i < close_i:
        raise ValueError
    if close_direct <= close_i:
        result = {"Parameters": parse_parametets(string[len(tag)+1:-2].split())}
    else:
        result["Parameters"] = parse_parametets(string[len(tag)+1 : close_i+1].split())
        content_string = string[close_i + 2 :]
        while content_string.startswith('<'):
            new_tag = content_string[1:].split()[0]
            _i = new_tag.find('>')
            _f = new_tag.find('<')
            if _i >1 and _i < _f:
                new_tag = new_tag[:_i]
            # print(f"new tag = {new_tag}")
            new_tag_string, char_number = extract_string_for_tag(content_string, new_tag)
            # print(f"new tag string {new_tag_string}")
            result[new_tag] = parse_tag(new_tag_string, new_tag)
            content_string = content_string[char_number:]
        if len(content_string)>0:
            result["Content"] = content_string
    
    return result

def parse_nmd(file_name):
    with open(file_name, 'rb') as f:
        string = f.read()
    full_string = string[string.find(b'<SAMPLE'):string.find(b'</SAMPLE>')+len('</SAMPLE>')].decode()
    result = {}
    w_string , f = extract_string_for_tag(full_string, 'SAMPLE')
    result['SAMPLE'] = parse_tag(w_string, 'SAMPLE')
    return result

In [18]:
class NanoIndForm:
    _result_xls_form_map = {
    'Target  Depth': 'Target depth [nm]',
    'Target  Load': 'Target load [mN]',
    'Depth To End Ave.': 'Start of averaging depth [nm]',
    'Depth To End Ave.': 'End of averaging depth [nm]',
    'YYYY_MM_DD': 'Measurement date',
    'Hold Maximum Load Time': 'Hold time at maximum load [s]',
    'RelHumidity': 'Relative humidity [%]',
    'EnvironmentalGas': 'Environmental gas',
    'MeasurementPos': 'Measurement position',
    'Temperature': 'Temperature [°C]',
    'Target Ind. Strain Rate': 'Target strain rate [/s]',
    # :'Target displacement rate [nm/s]'  # either this or ^ 
    }
    _nano_ind_scheme={'ID': {'required': True, 'options': []},
 'External/alias ID': {'required': False, 'options': []},
 'User': {'required': True, 'options': []},
 'Date': {'required': True, 'options': []},
 'Affiliation': {'required': False, 'options': []},
 'DOIs': {'required': False, 'options': []},
 'Temperature [°C]': {'required': False, 'options': []},
 'Relative humidity [%]': {'required': False, 'options': []},
 'Environmental gas': {'required': False, 'options': []},
 'Operator': {'required': False, 'options': []},
 'Instrument ID': {'required': False, 'options': []},
 'Sample ID': {'required': False, 'options': []},
 'Parent sample ID': {'required': False, 'options': []},
 'Any data set to be linked with this experiment': {'required': False,
  'options': []},
 'Environmental protection during sample processing': {'required': False,
  'options': []},
 'Pre-treatment': {'required': False, 'options': []},
 'Measurement position': {'required': False, 'options': []},
 'Sample orientation': {'required': False, 'options': []},
 'Type of test': {'required': False, 'options': []},
 'Control method': {'required': False, 'options': []},
 'Tip ID': {'required': False, 'options': []},
 'Diamond area function': {'required': False, 'options': []},
 'Date of calibration': {'required': False, 'options': []},
 'Frame stiffness [N/m]': {'required': False, 'options': []},
 'Target load [mN]': {'required': False, 'options': []},
 'Target depth [nm]': {'required': False, 'options': []},
 'Continuous stiffness measurement': {'required': False, 'options': []},
 'Drift correction enabled': {'required': False, 'options': []},
 'Sample temperature [°C]': {'required': False, 'options': []},
 'Tip temperature [°C]': {'required': False, 'options': []},
 'Target strain rate [/s]': {'required': False, 'options': []},
 'Target loading rate [mN/s]': {'required': False, 'options': []},
 'Target displacement rate [nm/s]': {'required': False, 'options': []},
 'Start of averaging depth [nm]': {'required': False, 'options': []},
 'End of averaging depth [nm]': {'required': False, 'options': []},
 'Hold time at maximum load [s]': {'required': False, 'options': []},
 'Measurement date': {'required': False, 'options': []},
 'Comments': {'required': False, 'options': []}}
    def __init__(self):
        for key, value in self._nano_ind_scheme.items():
            value['value'] = None
            
    def __getitem__(self, item):
        if item not in self.keys():
            raise KeyError(item)
        return self._nano_ind_scheme[item]['value']
    
    def __setitem__(self, item, value):
        if item not in self.keys():
            raise KeyError(item)
        self._nano_ind_scheme[item]['value'] = value
    
    def keys(self):
        return self._nano_ind_scheme.keys()
    
    def to_dict(self):
        return {key: self._nano_ind_scheme[key]['value'] for key in self._nano_ind_scheme}
    
    def items(self):
        return self.to_dict().items()
    
    def parse_xls_file(self, xls_file, row_name=1):
        self._xls_file = pd.ExcelFile(xls_file)
        result_sheet = pd.read_excel(self._xls_file, sheet_name='Results', header=[0,1])
        mask = (result_sheet['Test'] == row_name).values
        result_sheet = result_sheet[mask]
        for key, own_key in self._result_xls_form_map.items():
            try:
                res = result_sheet[key].values[0,0]
                if res == "0" or res == 0:
                    res = None
                self[own_key] = res
            except KeyError:
                self[own_key] = None
                
        
        m_date = self['Measurement date']
        if isinstance(m_date, (int, str,  np.int64)):
            year = int(str(m_date)[0:4])
            month = int(str(m_date)[4:6])
            day = int(str(m_date)[6:])
            self['Measurement date'] = date(year, month, day)
                  
    def parse_nmd(self, nmd_file):
        self.nmd_dict = parse_nmd(nmd_file)
        machine_config_dict = self.nmd_dict['SAMPLE']['MACHINECONFIG']["Parameters"]
        self['Frame stiffness [N/m]'] = machine_config_dict['FRAMESTIFFNESS']
        self['Diamond area function'] = self._parse_area_coeffs(machine_config_dict)
        print(machine_config_dict.keys())
    
    @staticmethod
    def _parse_area_coeffs(machine_config_dict):
        result = ""
        n_coeff = machine_config_dict['AREACOEFFS']
        for i in range(n_coeff):
            key = f"AREACOEFF{i}"
            result += f"{machine_config_dict[key]}x**{i} +"
        return result
        
    
    def __repr__(self):
        result = "Meta data scheme:    Nanoindentation \n"
        result+= 100 * "=" + '\n'
        for key, value in self.items():
            result += f"{key}".ljust(50) + f"  =  {value} ".ljust(50)  + "\n"
        return result
        

In [19]:
form = NanoIndForm()

In [20]:
form.parse_nmd('DummyData/Dummy_1.NMD')

dict_keys(['SAMPLE', 'NAME', 'CREATETIME', 'MODIFIEDTIME', 'EQUATION', 'COMMENT', 'SERIAL', 'TIPMODULUS', 'TIPPOISSON', 'FRAMESTIFFNESS', 'AREACONSTANT', 'AREACOEFFS', 'AREACOEFF0', 'AREACOEFF1', 'AREACOEFF2', 'AREACOEFF3', 'AREACOEFF4', 'EPSILON', 'BETA', 'LFCALCDEPTHMIN', 'LFCALCDEPTHMAX', 'TIPCALCDEPTHMIN', 'TIPCALCDEPTHMAX', 'SAMPLEMODULUS', 'SAMPLEPOISSON'])


In [25]:
form.parse_xls_file('DummyData/Dummy_1.xlsx')

In [26]:
form

Meta data scheme:    Nanoindentation 
ID                                                  =  None                                         
External/alias ID                                   =  None                                         
User                                                =  None                                         
Date                                                =  None                                         
Affiliation                                         =  None                                         
DOIs                                                =  None                                         
Temperature [°C]                                    =  21                                           
Relative humidity [%]                               =  None                                         
Environmental gas                                   =  None                                         
Operator                                            =

In [ ]:
# form.nmd_dict['SAMPLE']['Parameters']
#  'TEMPLATENAME': 'Test_Standardindent_Carl.NMT' > Config-file
#  form.nmd_dict['SAMPLE']['HWCONFIG']   >  federkonstanten von indenter selbst
#  form.nmd_dict['SAMPLE']['MACHINECONFIG'] > Tip metadaten, Kalibrier-daten (datum usw.) DAF >'TIPMODULUS': 1140, 'TIPPOISSON': 0.07, in tip schema!
# form.nmd_dict['SAMPLE']['TESTSEQUENCE'] Parser Error

In [13]:
result_sheet = pd.read_excel(form._xls_file, sheet_name='Results', header=[0,1])
test_sheets = [pd.read_excel(form._xls_file, sheet_name=sheet_name, header=[0,1]) for sheet_name in form._xls_file.sheet_names if sheet_name.startswith('Test')] 

In [9]:
result_sheet.keys()

MultiIndex([(                     'Test', 'Unnamed: 0_level_1'),
            (            'Target  Depth',                 'nm'),
            (                     'hmax',                 'nm'),
            (             'Target  Load',                 'mN'),
            (                     'Pmax',                 'mN'),
            (  'Target Ind. Strain Rate',                '%/s'),
            (            'Ave. HARDNESS',                'GPa'),
            (             'Ave. MODULUS',                'GPa'),
            (    'Ave. STIFFNESS^2/LOAD',                'GPa'),
            (               'Drift Rate',               'nm/s'),
            (                       'ID',             'String'),
            (             'InstrumentID',               'None'),
            (                   'Sample',              'Index'),
            (                 'SampleID',             'String'),
            (              'RelHumidity',               'None'),
            ('Poisson's R

In [46]:
ipysheet.from_dataframe(result_sheet)

Sheet(cells=(Cell(column_end=0, column_start=0, numeric_format=None, row_end=4, row_start=0, squeeze_row=False…

In [10]:
# poisson ration missing in meta data scheme

In [11]:
result_sheet

,Test,Target Depth,hmax,Target Load,Pmax,Target Ind. Strain Rate,Ave. HARDNESS,Ave. MODULUS,Ave. STIFFNESS^2/LOAD,Drift Rate,...,Vacuum,YYYY_MM_DD,Operator,MeasurementPos,ParentSampleID,Type,Hold Maximum Load Time,EnvironmentalGas,Depth To End Ave.,Depth To Start Ave.
,Unnamed: 0_level_1,nm,nm,mN,mN,%/s,GPa,GPa,GPa,nm/s,...,None,None,String,None,String,None,s,None,nm,nm
0,1,500,500.000,10,0.1,0.2,0.007000,-0.2000,5.0000,0,...,1,20220208,0,0,0,0,1,0,500,350
1,2,500,501.000,10,0.1,0.2,0.004000,-0.1000,7.8000,0,...,1,20220208,0,0,0,0,1,0,500,350
2,Average,500,500.700,10,0.1,0.2,0.005400,-0.1400,6.4000,0,...,1,20220208,0,0,0,0,1,0,500,350
3,Standard Deviation,0,0.300,0,0.0,0.0,0.001600,0.0100,1.3600,0,...,0,0,0,0,0,0,0,0,0,0
4,Coefficient of Variation,0,0.001,0,0.0,0.0,0.287361,-0.0502,0.2123,0,...,0,0,0,0,0,0,0,0,0,0


# TIFF

In [49]:
tiff_dir = 'FIB Metadaten Testbilder'
tiff_files = os.listdir(tiff_dir)

In [50]:
import os

In [51]:
from PIL import Image
from PIL.TiffTags import TAGS

In [52]:
with open(os.path.join(tiff_dir, tiff_files[0]), 'rb') as f:
    tiff_0_b = f.read()

In [53]:
tiff_0 = Image.open(os.path.join(tiff_dir, tiff_files[0]))

In [43]:
dict(tiff_0.tag_v2.items())

{256: 1536,
 257: 1103,
 258: (8, 8, 8),
 259: 1,
 262: 2,
 273: (8,
  4616,
  9224,
  13832,
  18440,
  23048,
  27656,
  32264,
  36872,
  41480,
  46088,
  50696,
  55304,
  59912,
  64520,
  69128,
  73736,
  78344,
  82952,
  87560,
  92168,
  96776,
  101384,
  105992,
  110600,
  115208,
  119816,
  124424,
  129032,
  133640,
  138248,
  142856,
  147464,
  152072,
  156680,
  161288,
  165896,
  170504,
  175112,
  179720,
  184328,
  188936,
  193544,
  198152,
  202760,
  207368,
  211976,
  216584,
  221192,
  225800,
  230408,
  235016,
  239624,
  244232,
  248840,
  253448,
  258056,
  262664,
  267272,
  271880,
  276488,
  281096,
  285704,
  290312,
  294920,
  299528,
  304136,
  308744,
  313352,
  317960,
  322568,
  327176,
  331784,
  336392,
  341000,
  345608,
  350216,
  354824,
  359432,
  364040,
  368648,
  373256,
  377864,
  382472,
  387080,
  391688,
  396296,
  400904,
  405512,
  410120,
  414728,
  419336,
  423944,
  428552,
  433160,
  437768,
  44

In [39]:
meta_dict = {TAGS[key]: tiff_0.tag[key] for key in tiff_0.tag_v2.keys()}

KeyError: 34682

In [58]:
def parse_tiff_metadata(tiff_img):
    result=[]
    for key, val in tiff_img.tag_v2.items():
        if isinstance(val, str):
            result.append(val)

    result_2 = {'None': {}}
    key =  'None'
    
    for str_entry in result:
        for line in str_entry.replace('\r', '').split('\n'):
            if len(line)>2 and line[0] == '[' and line[-1] == ']':
                key = line[1:-1]
                result_2[key] = {}
            else:
                sub_key = line.split('=')[0]
                value = '='.join(line.split('=')[1:])
                result_2[key][sub_key] = value
    
    if isinstance(result_2['None'], dict) and len(result_2['None']) == 0:
        del result_2['None']
    for val in result_2.values():
        if isinstance(val, dict) and '' in val and val[''] == '':
            del val['']
    return result_2

In [59]:
parse_tiff_metadata(tiff_0)

{'User': {'Date': '02/03/2022',
  'Time': '05:47:36 PM',
  'User': 'supervisor',
  'UserText': 'IMM Helios 600i',
  'UserTextUnicode': '49004D004D002000480065006C0069006F00730020003600300030006900'},
 'System': {'Type': 'DualBeam',
  'Dnumber': '9922804',
  'Software': '5.5.1.3318',
  'BuildNr': '3318',
  'Source': 'FEG',
  'Column': 'Elstar',
  'FinalLens': 'Elstar',
  'Chamber': 'xT-SDB',
  'Stage': '6inch',
  'Pump': 'TMP',
  'ESEM': 'no',
  'Aperture': 'AVA',
  'Scan': 'PIA 3.0',
  'Acq': 'PIA 3.0',
  'EucWD': '0.004',
  'SystemType': 'Helios NanoLab" 600i',
  'DisplayWidth': '0.518',
  'DisplayHeight': '0.324'},
 'Beam': {'HV': '10000',
  'Spot': '-1',
  'StigmatorX': '-0.000931353',
  'StigmatorY': '-0.0073799',
  'BeamShiftX': '0',
  'BeamShiftY': '0',
  'ScanRotation': '0',
  'ImageMode': 'Normal',
  'FineStageBias': '',
  'Beam': 'EBeam',
  'Scan': 'EScan'},
 'EBeam': {'Source': 'FEG',
  'ColumnType': 'Elstar',
  'FinalLens': 'Elstar',
  'Acq': 'PIA 3.0',
  'Aperture': 'AVA',


In [48]:
from datetime import datetime

In [50]:
a = datetime.now()

In [51]:
a

datetime.datetime(2022, 3, 4, 10, 35, 4, 939860)

In [53]:
a.isoformat()

'2022-03-04T10:35:04.939860'

In [54]:
a

datetime.datetime(2022, 3, 4, 10, 35, 4, 939860)

In [55]:
datetime.fromisoformat(a.isoformat())

datetime.datetime(2022, 3, 4, 10, 35, 4, 939860)